In [1]:
from selenium import webdriver

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys 
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.action_chains import ActionChains

from time import sleep as para
from time import time as contar
import datetime as dt

import pandas as pd
import os
import re

from openpyxl import load_workbook, Workbook

In [2]:
class Navegacao():
    def __init__(self):
        options=webdriver.ChromeOptions()
        if input('Deseja esconder o navegador? y/n\n')=='y': 
            options.add_argument("--headless")
            self.navegador = webdriver.Chrome(chrome_options=options)
        else: self.navegador = webdriver.Chrome()
        self.vars = {'contas': [],
                    'link do afim': '',
                    'ignored_exceptions': (NoSuchElementException,StaleElementReferenceException), 
                    'senha': f'Jaspekde27.{dt.date.today().month*3}'}
        self.pl = {}

    def espera_login(self, ano):
        if ano > 2018:
            self.navegador.find_element(By.ID, 'user').send_keys('034.410.422-20')
            self.navegador.find_element(By.ID, 'password').send_keys(self.vars['senha'])
            self.navegador.find_element(By.ID, 'password').send_keys(Keys.ENTER)
        else:
            self.navegador.find_element(By.NAME, 'user').send_keys('03441042220')
            self.navegador.find_element(By.NAME, 'password').send_keys(self.vars['senha'])
            self.navegador.find_element(By.NAME, 'password').send_keys(Keys.ENTER)

    def esperar_elementos_carregarem(self, tipo_elemento: str, nome_elemento: str, varios_elementos: bool = False, tempo_maximo_espera: int = 30):
        """Espera um ou vários elementos carregarem na página

        Args:
            tipo_elemento (str): tipo do elemento
            nome_elemento (str): nome do elemento
            varios_elementos (bool): procurar mais de um elemento
            tempo_maximo_espera (int): tempo determinado para executar a espera, em segundos. Padrão 30 (0,5 min).
        """    
        tempo_inicial = contar() 
        
        if varios_elementos:
            elementos = WebDriverWait(self.navegador, tempo_maximo_espera, ignored_exceptions=self.vars['ignored_exceptions'])\
                            .until(expected_conditions.presence_of_all_elements_located((tipo_elemento, nome_elemento)))
            tempo_final = contar()
            print(f'>>> Elementos "{nome_elemento}" encontrados em {round(tempo_final - tempo_inicial, 2)}s! (len({len(elementos)}))')
            return elementos
        else:
            elemento = WebDriverWait(self.navegador, tempo_maximo_espera, ignored_exceptions=self.vars['ignored_exceptions'])\
                            .until(expected_conditions.presence_of_element_located((tipo_elemento, nome_elemento)))
            tempo_final = contar()
            if len(elemento.text) > 20: elemento_texto_resumido = elemento.text[:20]
            else: elemento_texto_resumido = elemento.text
                
            print(f'>>> Elemento "{elemento_texto_resumido}" encontrado em {round(tempo_final - tempo_inicial, 2)}s!')
            return elemento

    def entra_AFIM(self, afim):
        """Abre o AFIM

        Args:
            afim (_type_): Ano no qual o FIM irá abrir
        """
        print(f'\n=============== Executando entra_AFIM {afim}! ===============')
        
        if afim <= 2018:
            link = f'http://afim2.manaus.am.gov.br/AfimPRD{afim}/'
            self.navegador.get(link)
            self.espera_login(afim)
            para(1)
            self.navegador.switch_to.window(self.navegador.window_handles[-1])

        #AFIM DE 2019
        if afim == 2019:
            link = 'https://afim1.manaus.am.gov.br/AfimPRD2019/'
            self.navegador.get(link)
            self.espera_login(afim)
            para(2)
            for w in self.navegador.window_handles:
                self.navegador.switch_to.window(w)
                if 'mensagem' in self.navegador.current_url: self.navegador.close()
                elif str(afim) in self.navegador.current_url: pass
                else: self.navegador.close()
                
            self.navegador.switch_to.window(self.navegador.window_handles[-1])

        if afim >= 2020:
            link = f'https://afim1.manaus.am.gov.br/AfimPRD{afim}/'
            self.navegador.get(link)
            
            self.espera_login(afim)
            self.navegador.switch_to.window(self.navegador.window_handles[-1])
            para(1)

        self.vars['link do afim'] = link

        print(f'=============== Finalizando entra_AFIM {afim}! ===============\n')
        para(2)

    def NL_detalhada(self, obj_NL):
        """Pesquisa a Nota de Lançamento desejada para retirar as informações

        Args:
            obj_NL (_type_): Objeto Selenium da Nota de Lançamento

        Returns:
            _type_: Tupla com as informações extraidas
        """
        if not 'NL' in obj_NL.text: 
            print(f'\n=============== Impossível executar! Não é uma NL ({obj_NL.text}) ===============')
            return ('', '', '', '', '', '','')
        
        
        print(f'\n=============== Executando a NL ({obj_NL.text}) ===============')
        para(2)
        self.navegador.switch_to.window(self.navegador.window_handles[-1])

        elemento = obj_NL
        ActionChains(self.navegador)\
        .double_click(elemento)\
        .perform()

        para(1)
        self.navegador.switch_to.window(self.navegador.window_handles[-1])
        
        credor_empresa = self.esperar_elementos_carregarem(By.CSS_SELECTOR, 'tr td input[name="txtCredor"]').get_attribute('value')
        obs = self.esperar_elementos_carregarem(By.CSS_SELECTOR, 'textarea[name="txtObservacao"]').text
        NE = self.esperar_elementos_carregarem(By.CSS_SELECTOR, 'input[name="txtNuNe"').get_attribute('value')
        natureza_despesa = self.esperar_elementos_carregarem(By.CSS_SELECTOR, 'input[name="txtRdNaturezaDespesa"]').get_attribute('value')
        doc = self.esperar_elementos_carregarem(By.CSS_SELECTOR, 'input[name="txtRdNuDocmento"').get_attribute('value')
        processo = self.esperar_elementos_carregarem(By.CSS_SELECTOR, 'input[name="txtRdNuProcesso"').get_attribute('value')
        
        self.navegador.close()
        self.navegador.switch_to.window(self.navegador.window_handles[-1])
        
        return (obj_NL.text.replace(' ', ''), credor_empresa, obs, NE, natureza_despesa, doc, processo)
    
    def RAZAO(self, ano):
        print(f'\n=============== Executando RAZAO {ano}! ===============')

        # Ano anterior a 2020
        lisne_link = self.vars['link do afim'] + 'Razao.do'
        self.navegador.get(lisne_link)

        para(1)
        self.navegador.switch_to.window(self.navegador.window_handles[-1])
        self.vars['contas'] = [cbo.get_attribute('Value') for cbo in self.navegador.find_elements(By.CSS_SELECTOR, "select[name='cboConta'] option[value^='1232105']")]
        para(1)

    def RAZAO_detalhada_depois_de_2020(self, ano, conta):

        print(f'Selecionando a Data Inicial: 01/01/{ano}')
        if self.esperar_elementos_carregarem(By.ID, 'mkdDataInicio').get_attribute('value')!=f'01/01/{ano}':
            self.esperar_elementos_carregarem(By.ID, 'mkdDataInicio').click()
            self.esperar_elementos_carregarem(By.CSS_SELECTOR, "li[data-view='month current']").click()
            self.esperar_elementos_carregarem(By.CSS_SELECTOR, "ul li[data-view='month']", True, 30)[0].click()
        else: print('Data Inicial já selecionada')
            
        print(f'Selecionando a Data Final: 31/12/{ano}')
        if self.esperar_elementos_carregarem(By.ID, 'mkdDataFim').get_attribute('value')!=f'31/12/{ano}':
            self.esperar_elementos_carregarem(By.ID, 'mkdDataFim').click()
            self.esperar_elementos_carregarem(By.CSS_SELECTOR, "div[class*='datepicker-top-right'] li[data-view='month current']").click()
            self.esperar_elementos_carregarem(By.CSS_SELECTOR, "div[class*='datepicker-top-right'] ul[data-view='months'] li[data-view='month']", True, 30)[-1]
        else: print('Data Final já selecionada')

        # Implementar aqui

    def RAZAO_detalhada_antes_de_2020(self, ano, conta):
        print(f'\n=============== Executando RAZAO_detalhada em {ano}! ===============')
        ano = str(ano)
        N = self

        print(f'Selecionando a Data Inicial: 01/01/{ano}')
        if self.esperar_elementos_carregarem(By.ID, 'mkdDataInicio').get_attribute('value')!=f'01/01/{ano}':
            self.esperar_elementos_carregarem(By.ID, 'img0').click()
            while not (ano in self.navegador.find_elements(By.CSS_SELECTOR, "font[size='1']")[0].text): self.navegador.find_element(By.ID, "toLeft").click()
            if self.navegador.find_element(By.CSS_SELECTOR, "#month [selected]").text!='Janeiro': self.esperar_elementos_carregarem(By.CSS_SELECTOR, "#month").send_keys('Janeiro')
            self.esperar_elementos_carregarem(By.CSS_SELECTOR, "td[class='calReg']", True, 30)[0].click()
        else: print('Data Inicial já selecionada')

        print(f'Selecionando a Data Final: 31/12/{ano}')
        if self.esperar_elementos_carregarem(By.ID, 'mkdDataFim').get_attribute('value')!=f'31/12/{ano}':
            self.esperar_elementos_carregarem(By.ID, 'img1').click()
            while not (ano in self.navegador.find_elements(By.CSS_SELECTOR, "font[size='1']")[0].text): self.navegador.find_element(By.ID, "toLeft").click()
            if self.navegador.find_element(By.CSS_SELECTOR, "#month [selected]").text!='Dezembro': self.esperar_elementos_carregarem(By.CSS_SELECTOR, "#month").send_keys('Dezembro')
            self.esperar_elementos_carregarem(By.CSS_SELECTOR, "td[class='calReg']", True, 30)[-1].click()
        else: print('Data Final já selecionada')

        self.navegador.switch_to.window(self.navegador.window_handles[-1])

        self.esperar_elementos_carregarem( By.CSS_SELECTOR, 'select[name="cboUnidadeGestora"]').send_keys("270101 - Secretaria Municipal de Infraestrutura")
        self.navegador.find_element(By.CSS_SELECTOR, 'select[name="cboUnidadeGestora"]').click()

        self.esperar_elementos_carregarem(By.NAME, 'cboConta').send_keys(conta)

        N.esperar_elementos_carregarem(By.CSS_SELECTOR, 'input[name="..."]').click()
        try:
            alerta = N.navegador.switch_to.alert
            alerta.accept()
            ccs = []
        except: ccs = [i.text for i in N.esperar_elementos_carregarem(By.CSS_SELECTOR, 'select[id="lstHelp"] option', True)]

        for c in ccs:
            N.esperar_elementos_carregarem(By.CSS_SELECTOR, 'input[name="..."]').click()
            element = N.esperar_elementos_carregarem(By.CSS_SELECTOR, f'select[id="lstHelp"] option[value="{c}"]')
            ActionChains(N.navegador)\
                .double_click(element)\
                .perform()

            conta_nominada = N.navegador.find_element(By.CSS_SELECTOR, f"select[name='cboConta'] option[value='{conta}']").text + f' ({c})'

            para(1)
            try: N.esperar_elementos_carregarem(By.ID, 'IMGprocurar').click()
            except: N.esperar_elementos_carregarem(By.ID, 'btnProcurar').click()

            if '\\' in conta_nominada: conta_nominada = conta_nominada.replace('\\','/')

            lista_no_razao = []
            # Tabela
            for n in N.navegador.find_elements(By.CSS_SELECTOR, '#dbGrid tr'): lista_no_razao.append(n.text.split('  '))

            detalhes_da_NL = [['NOTA DE LANÇAMENTO','CREDOR','OBSERVAÇÃO','NOTA DE EMPENHO',
            'NATUREZA DE DESPESA','N° DO DOCUMENTO', 'PROCESSO']]

            for i in N.navegador.find_elements(By.CSS_SELECTOR, '#dbGrid tr td[onclick="selectCell(this,3)"]'): detalhes_da_NL.append(N.NL_detalhada(i))

            temp = []
            for indice,linha in enumerate(lista_no_razao):
                print(f'{indice}: {linha} + {detalhes_da_NL[indice-1]}') 
                linha.extend(detalhes_da_NL[indice])
                temp.append(linha)
                self.pl[conta_nominada] = lista_no_razao

            # pl.save(f'123210500\\123210500-BENS DE USO COMUM DO POVO - {ano}.xlsx')
            
            self.pl[conta_nominada][1][0] = f'01/01/{ano}'
            self.pl[conta_nominada][1][6] = 'Saldo Anterior'
            self.pl[conta_nominada][1][5] = ''
            self.pl[conta_nominada][-1][0] = f'31/12/{ano}'
            self.pl[conta_nominada][-1][6] = 'Saldo Atual'
            self.pl[conta_nominada][-1][5] = ''

            cabecalho = self.pl[conta_nominada][0]
            self.pl[conta_nominada] = pd.DataFrame(self.pl[conta_nominada][1:], columns=cabecalho)
            # self.pl[conta_nominada].columns = cabecalho

        print('=============== Finalizando RAZAO! ===============')
        # return f'123210500\\123210500-BENS DE USO COMUM DO POVO - {ano}.xlsx'
        
    def contas_unidas(self, ano):
        for pl in self.pl:
            self.pl[pl].insert(0, "Conta Contábil", [pl for i in range(len(self.pl[pl]))],True)
            display(self.pl[pl][:1])
            
        result = pd.concat([self.pl[pl] for pl in self.pl])
        path = f"123210500\\123210500-BENS DE USO COMUM DO POVO - {ano}.xlsx"

        print(f'Salvando as contas em uma planilha excel em "{path}"')
        result.to_excel(path, index=False)

    def balancete(self):
        print(f'=============== Inicializando balancete! ===============\n')
        # if os.path.exists(r):
        #     print("O arquivo já existe.")
        #     return 0

        balancete_link = self.vars['link do afim'] + 'Balancete.do'
        self.navegador.get(balancete_link)

        para(1)
        self.navegador.switch_to.window(self.navegador.window_handles[-1])

        self.vars['contas'] = [l.get_attribute('value') for l in self.navegador.find_elements(By.CSS_SELECTOR, '[value^="1232105"]')]

        # clicando no radio "Unidade Gestora/Gestão"
        self.esperar_elementos_carregarem(By.CSS_SELECTOR, 'input[value="2"]').click()

        self.navegador.find_element(By.CSS_SELECTOR, 'select[name="cboUnidadeGestora"]').send_keys('270101 - Secretaria Municipal de Infraestrutura')
        self.navegador.find_element(By.CSS_SELECTOR, 'option[value="00001"]').click()
        self.navegador.find_element(By.CSS_SELECTOR, 'select[name="cboMes"] option[value="12"]').click()
        self.navegador.find_element(By.CSS_SELECTOR, 'select[name="cboValores"] option[value="A"]').click()

        self.navegador.find_element(By.CSS_SELECTOR, f'select[name="cboContaIni"] option[value="{self.vars["contas"][0]}"]').click()
        self.navegador.find_element(By.CSS_SELECTOR, f'select[name="cboContaFim"] option[value="{self.vars["contas"][-1]}"]').click()

        self.navegador.find_element(By.CSS_SELECTOR, 'select[name="cboTipo"] option[value="2"]').click()
        if not self.navegador.find_element(By.CSS_SELECTOR, 'input[name="ssoOpcao"]').is_selected(): self.navegador.find_element(By.CSS_SELECTOR, 'input[name="ssoOpcao"]').click()

        self.navegador.find_element(By.CSS_SELECTOR, '[id="IMGprocurar"]').click()
        para(3)
        self.extraindo_balancete()
        print(f'=============== Finalizando balancete! ===============\n')

    def extraindo_balancete(self):
        print('========== Executando extraindo_balancete! ==========')
        balancete = {}
        for i in [e.text for e in self.navegador.find_elements(By.CSS_SELECTOR, 'tr > th.label') if e.text!='']: balancete[i] = []

        keys = list(balancete.keys())

        linhas = [l for l in self.navegador.find_elements(By.CSS_SELECTOR, 'tr tr[align]')]
        for linha in linhas: 
            l = [l for l in linha.text.split('  ') if l!='']
            # print('l:',l)
            if '1232105' in l[0]: 
                conta = l[0]
                # print('var conta:',conta)
            elif not '-' in l[0]:
                balancete[keys[0]].append(f"{conta} ({l[0].replace(' ', '')})")
                for index in range(1,6): balancete[keys[index]].append(int(l[index].replace('.', '').replace(',','')))
                balancete[keys[6]].append(l[6])
                # print('=== contas ===')
                # display(balancete)
        # print(len(contas))
        self.vars['balancete'] = pd.DataFrame(balancete)
        display(self.vars['balancete'])
        print('========== Finalizando extraindo_balancete! ==========')



In [3]:
ano = 2014
N = Navegacao()
N.entra_AFIM(ano)
N.balancete()
N.RAZAO(ano)
self = N


=============== Executando entra_AFIM 2014! ===============
=============== Finalizando entra_AFIM 2014! ===============

=============== Inicializando balancete! ===============

>>> Elemento "" encontrado em 0.01s!
========== Executando extraindo_balancete! ==========


,Conta Contábil,Saldo Inicial,Saldo Mês Anterior,Debito Até Mês,Credito Até Mês,Saldo Atual,D/C
0,123210501-ifraestrutura serv. terceiros (@),0,79171217,87291217,8120000,79171217,D
1,123210502-Ruas (@),0,7401046801,8498044603,1503152155,6994892448,D
2,123210503-Infraestrutura : massa asfalta (@),0,1036065789,1107559541,75966231,1031593310,D
3,123210504-Sistema De Drenagem (@),0,2097526439,2381277488,147238469,2234039019,D
4,123210517-Obras de Infraestrutura de revitali...,0,2180000,2180000,0,2180000,D
5,123210517-Obras de Infraestrutura de revitali...,0,425044583,425044583,0,425044583,D
6,123210517-Obras de Infraestrutura de revitali...,14375482160,22642780499,10306677421,1694389646,22987769935,D
7,123210519-Obras de Urbanização e Consrvação d...,0,4744950549,5456595838,515034081,4941561757,D
8,123210520-Obras de Infra estrutura para Copa ...,0,4705806333,4719176316,0,4719176316,D
9,123210580-Estudos e Projetos P/ Infraestrutur...,0,227832100,553170303,207300049,345870254,D


========== Finalizando extraindo_balancete! ==========
=============== Finalizando balancete! ===============


=============== Executando RAZAO 2014! ===============


Selecionando as datas (Funcional)

In [4]:
print(f'Selecionando a Data Inicial: 01/01/{ano}')
if self.esperar_elementos_carregarem(By.ID, 'mkdDataInicio').get_attribute('value')!=f'01/01/{ano}':
    self.esperar_elementos_carregarem(By.ID, 'img0').click()
    while not (str(ano) in self.navegador.find_elements(By.CSS_SELECTOR, "font[size='1']")[0].text): self.navegador.find_element(By.ID, "toLeft").click()
    if self.navegador.find_element(By.CSS_SELECTOR, "#month [selected]").text!='Janeiro': self.esperar_elementos_carregarem(By.CSS_SELECTOR, "#month").send_keys('Janeiro')
    self.esperar_elementos_carregarem(By.CSS_SELECTOR, "td[class='calReg']", True, 30)[0].click()
else: print('Data Inicial já selecionada')

print(f'Selecionando a Data Final: 31/12/{ano}')
if self.esperar_elementos_carregarem(By.ID, 'mkdDataFim').get_attribute('value')!=f'31/12/{ano}':
    self.esperar_elementos_carregarem(By.ID, 'img1').click()
    while not (str(ano) in self.navegador.find_elements(By.CSS_SELECTOR, "font[size='1']")[0].text): self.navegador.find_element(By.ID, "toLeft").click()
    if self.navegador.find_element(By.CSS_SELECTOR, "#month [selected]").text!='Dezembro': self.esperar_elementos_carregarem(By.CSS_SELECTOR, "#month").send_keys('Dezembro')
    self.esperar_elementos_carregarem(By.CSS_SELECTOR, "td[class='calReg']", True, 30)[-1].click()
else: print('Data Final já selecionada')

Selecionando a Data Inicial: 01/01/2014
>>> Elemento "" encontrado em 0.01s!
>>> Elemento "" encontrado em 0.01s!
>>> Elementos "td[class='calReg']" encontrados em 0.01s! (len(31))
Selecionando a Data Final: 31/12/2014
>>> Elemento "" encontrado em 0.01s!
>>> Elemento "" encontrado em 0.01s!
>>> Elemento "Janeiro
Fevereiro
Ma" encontrado em 0.01s!
>>> Elementos "td[class='calReg']" encontrados em 0.01s! (len(31))


In [5]:
for c in range(len(self.vars['balancete'])):
    conta_contabil = re.search(r' (\d.*) \(', self.vars['balancete'].iloc[c,0]).group(1)
    conta_corrente = re.search(r'\s\((.*)\)', self.vars['balancete'].iloc[c,0]).group(1)

    print(conta_contabil, conta_corrente)

    self.esperar_elementos_carregarem( By.CSS_SELECTOR, 'select[name="cboUnidadeGestora"]').send_keys("270101 - Secretaria Municipal de Infraestrutura")
    self.navegador.find_element(By.CSS_SELECTOR, 'select[name="cboUnidadeGestora"]').click()

    self.esperar_elementos_carregarem(By.NAME, 'cboConta').send_keys(conta_contabil)
    self.esperar_elementos_carregarem(By.CSS_SELECTOR, '#txtContaCorrente').send_keys(conta_corrente)

    try: N.esperar_elementos_carregarem(By.ID, 'IMGprocurar').click()
    except: N.esperar_elementos_carregarem(By.ID, 'btnProcurar').click()

    lista_no_razao = []
    # Tabela
    for n in N.navegador.find_elements(By.CSS_SELECTOR, '#dbGrid tr'): lista_no_razao.append(n.text.split('  '))

    lista_no_razao = list(zip(*lista_no_razao))
    # nomes_colunas = [tupla[0] for tupla in lista_no_razao]

    # Criar o DataFrame a partir da lista de tuplas zip, especificando os nomes das colunas
    df = pd.DataFrame(lista_no_razao).transpose()

    df = df.set_axis(df.iloc[0], axis=1, copy=False)
    df = df.drop(len(df)-1).drop(0).drop(1)
    df = df.reset_index(drop=True)

    detalhes_da_NL = [['Número','CREDOR','OBSERVAÇÃO','NOTA DE EMPENHO',
            'NATUREZA DE DESPESA','N° DO DOCUMENTO', 'PROCESSO']]

    for i in self.navegador.find_elements(By.CSS_SELECTOR, '#dbGrid tr td[onclick="selectCell(this,3)"]'): 
        if i.text!=' ': detalhes_da_NL.append(self.NL_detalhada(i))

    df2 = pd.DataFrame(list(zip(*detalhes_da_NL))).transpose()

    # Atualizar os rótulos das colunas do DataFrame df2
    df2.columns = df2.iloc[0][:len(df2.columns)]

    # # Remover a primeira linha do DataFrame df2
    # df2 = df2.iloc[1:].reset_index(drop=True)

    # df2 = df2.set_axis(df.iloc[0], axis=1, copy=False)
    # df2 = df2.drop(0)
    # df2 = df2.reset_index(drop=True)

    print(self.vars['balancete'].iloc[0,0])
    display(df2)
    display(df)

    df_mescle = df2.merge(df, on='Número')
    self.vars[self.vars['balancete'].iloc[c,0]] = df_mescle
    display(self.vars[self.vars['balancete'].iloc[c,0]])


123210501-ifraestrutura serv. terceiros @
>>> Elemento "270101-SECRETARIA MU" encontrado em 0.01s!
>>> Elemento "111110100-Caixa
1111" encontrado em 0.01s!
>>> Elemento "" encontrado em 0.01s!
>>> Elemento "" encontrado em 0.01s!

=============== Executando a NL ( 2014NL00687) ===============
>>> Elemento "" encontrado em 0.01s!
>>> Elemento "Reconhecimento de De" encontrado em 0.01s!
>>> Elemento "" encontrado em 0.01s!
>>> Elemento "" encontrado em 0.01s!
>>> Elemento "" encontrado em 0.01s!
>>> Elemento "" encontrado em 0.01s!

=============== Executando a NL ( 2014NL02323) ===============
>>> Elemento "" encontrado em 0.01s!
>>> Elemento "Reconhecimento de De" encontrado em 0.0s!
>>> Elemento "" encontrado em 0.0s!
>>> Elemento "" encontrado em 0.01s!
>>> Elemento "" encontrado em 0.0s!
>>> Elemento "" encontrado em 0.0s!

=============== Executando a NL ( 2014NL02567) ===============
>>> Elemento "" encontrado em 0.01s!
>>> Elemento "Reconhecimento de De" encontrado em 0.0s!
>>> E

,Número,CREDOR,OBSERVAÇÃO,NOTA DE EMPENHO,NATUREZA DE DESPESA,N° DO DOCUMENTO,PROCESSO
0,Número,CREDOR,OBSERVAÇÃO,NOTA DE EMPENHO,NATUREZA DE DESPESA,N° DO DOCUMENTO,PROCESSO
1,2014NL00687,01216705000103 - TRANSENO DA AMAZ.REP.COM.TRAN...,Reconhecimento de Despesa PROC. Nº. 2014/2287/...,2014NE00302,44903901 - Outros Servicos de Terceiros - Pess...,1593,20142287290800007
2,2014NL02323,01057727000178 - LAGHI ENGENHARIA LTDA,Reconhecimento de Despesa PROC. 2014/17428/175...,2014NE00496,44903901 - Outros Servicos de Terceiros - Pess...,681,2013112171124200054
3,2014NL02567,01057727000178 - LAGHI ENGENHARIA LTDA,Reconhecimento de Despesa PROC.2014/11217/1680...,2014NE00471,44903901 - Outros Servicos de Terceiros - Pess...,675,2013112171124200046
4,2014NL03443,01057727000178 - LAGHI ENGENHARIA LTDA,Reconhecimento de Despesa PROC. 2014/11217/168...,2014NE00471,44903901 - Outros Servicos de Terceiros - Pess...,718,2013112171124200046
5,2014NL03834,34570556000135 - BAKER TILLY BRASIL-NORTE SS-AUD,Reconhecimento de Despesa PROC.2013/11217/1123...,2014NE01441,44903901 - Outros Servicos de Terceiros - Pess...,252,2013112171123200019
6,2014NL03951,34570556000135 - BAKER TILLY BRASIL-NORTE SS-AUD,Reconhecimento de Despesa PROC.2013/11217/1123...,2014NE01441,44903901 - Outros Servicos de Terceiros - Pess...,252,2013112171123200019
7,2014NL03977,34570556000135 - BAKER TILLY BRASIL-NORTE SS-AUD,fonte incorreta,2014NE01441,44903901 - Outros Servicos de Terceiros - Pess...,252,2013112171123200019
8,2014NL03978,34570556000135 - BAKER TILLY BRASIL-NORTE SS-AUD,Reconhecimento de Despesa PROC.2013/11217/1123...,2014NE01580,44903901 - Outros Servicos de Terceiros - Pess...,252,2013112171123200019
9,2014NL07156,34570556000135 - BAKER TILLY BRASIL-NORTE SS-AUD,Liquidação da NE nº 2014NE01441 REGULARIZAÇA...,,,,


,Data,UG,Gestão,Número,Evento,Movimento,D/C,Saldo,D/C
0,19/03/2014,270101,00001,2014NL00687,570406,"156.842,11",D,"156.842,11",D
1,20/06/2014,270101,00001,2014NL02323,570406,"111.752,52",D,"268.594,63",D
2,03/07/2014,270101,00001,2014NL02567,570406,"206.687,54",D,"475.282,17",D
3,22/08/2014,270101,00001,2014NL03443,570406,"275.830,00",D,"751.112,17",D
4,02/09/2014,270101,00001,2014NL03834,570406,"40.600,00",D,"791.712,17",D
5,04/09/2014,270101,00001,2014NL03951,570406,"40.600,00",D,"832.312,17",D
6,04/09/2014,270101,00001,2014NL03977,575406,"40.600,00",C,"791.712,17",D
7,08/09/2014,270101,00001,2014NL03978,570406,"40.600,00",D,"832.312,17",D
8,03/11/2014,270101,00001,2014NL07156,575406,"40.600,00",C,"791.712,17",D


,Número,CREDOR,OBSERVAÇÃO,NOTA DE EMPENHO,NATUREZA DE DESPESA,N° DO DOCUMENTO,PROCESSO,Data,UG,Gestão,Evento,Movimento,D/C,Saldo,D/C
0,2014NL00687,01216705000103 - TRANSENO DA AMAZ.REP.COM.TRAN...,Reconhecimento de Despesa PROC. Nº. 2014/2287/...,2014NE00302,44903901 - Outros Servicos de Terceiros - Pess...,1593,20142287290800007,19/03/2014,270101,00001,570406,"156.842,11",D,"156.842,11",D
1,2014NL02323,01057727000178 - LAGHI ENGENHARIA LTDA,Reconhecimento de Despesa PROC. 2014/17428/175...,2014NE00496,44903901 - Outros Servicos de Terceiros - Pess...,681,2013112171124200054,20/06/2014,270101,00001,570406,"111.752,52",D,"268.594,63",D
2,2014NL02567,01057727000178 - LAGHI ENGENHARIA LTDA,Reconhecimento de Despesa PROC.2014/11217/1680...,2014NE00471,44903901 - Outros Servicos de Terceiros - Pess...,675,2013112171124200046,03/07/2014,270101,00001,570406,"206.687,54",D,"475.282,17",D
3,2014NL03443,01057727000178 - LAGHI ENGENHARIA LTDA,Reconhecimento de Despesa PROC. 2014/11217/168...,2014NE00471,44903901 - Outros Servicos de Terceiros - Pess...,718,2013112171124200046,22/08/2014,270101,00001,570406,"275.830,00",D,"751.112,17",D
4,2014NL03834,34570556000135 - BAKER TILLY BRASIL-NORTE SS-AUD,Reconhecimento de Despesa PROC.2013/11217/1123...,2014NE01441,44903901 - Outros Servicos de Terceiros - Pess...,252,2013112171123200019,02/09/2014,270101,00001,570406,"40.600,00",D,"791.712,17",D
5,2014NL03951,34570556000135 - BAKER TILLY BRASIL-NORTE SS-AUD,Reconhecimento de Despesa PROC.2013/11217/1123...,2014NE01441,44903901 - Outros Servicos de Terceiros - Pess...,252,2013112171123200019,04/09/2014,270101,00001,570406,"40.600,00",D,"832.312,17",D
6,2014NL03977,34570556000135 - BAKER TILLY BRASIL-NORTE SS-AUD,fonte incorreta,2014NE01441,44903901 - Outros Servicos de Terceiros - Pess...,252,2013112171123200019,04/09/2014,270101,00001,575406,"40.600,00",C,"791.712,17",D
7,2014NL03978,34570556000135 - BAKER TILLY BRASIL-NORTE SS-AUD,Reconhecimento de Despesa PROC.2013/11217/1123...,2014NE01580,44903901 - Outros Servicos de Terceiros - Pess...,252,2013112171123200019,08/09/2014,270101,00001,570406,"40.600,00",D,"832.312,17",D
8,2014NL07156,34570556000135 - BAKER TILLY BRASIL-NORTE SS-AUD,Liquidação da NE nº 2014NE01441 REGULARIZAÇA...,,,,,03/11/2014,270101,00001,575406,"40.600,00",C,"791.712,17",D


123210502-Ruas @
>>> Elemento "270101-SECRETARIA MU" encontrado em 0.01s!
>>> Elemento "111110100-Caixa
1111" encontrado em 0.01s!
>>> Elemento "" encontrado em 0.01s!
>>> Elemento "" encontrado em 0.01s!

=============== Executando a NL ( 2014NL00393) ===============
>>> Elemento "" encontrado em 0.01s!
>>> Elemento "Reconhecimento de De" encontrado em 0.01s!
>>> Elemento "" encontrado em 0.01s!
>>> Elemento "" encontrado em 0.01s!
>>> Elemento "" encontrado em 0.01s!
>>> Elemento "" encontrado em 0.01s!

=============== Executando a NL ( 2014NL00468) ===============
>>> Elemento "" encontrado em 0.01s!
>>> Elemento "Reconhecimento de De" encontrado em 0.01s!
>>> Elemento "" encontrado em 0.0s!
>>> Elemento "" encontrado em 0.0s!
>>> Elemento "" encontrado em 0.0s!
>>> Elemento "" encontrado em 0.0s!

=============== Executando a NL ( 2014NL00501) ===============
>>> Elemento "" encontrado em 0.01s!
>>> Elemento "Reconhecimento de De" encontrado em 0.01s!
>>> Elemento "" encontrado em

#### MARKA 

In [ ]:
for ano in range(2020, 2023):
    N = Navegacao()
    N.entra_AFIM(str(ano))
    N.RAZAO(ano)
    for conta in N.vars['conta_contabil']: N.RAZAO_detalhada(ano, conta)
    N.navegador.close()
    N.contas_unidas(ano)


In [ ]:
self = N

self.esperar_elementos_carregarem( By.CSS_SELECTOR, 'select[name="cboUnidadeGestora"]').send_keys("270101 - Secretaria Municipal de Infraestrutura")
# self.navegador.find_element(By.CSS_SELECTOR, 'select[name="cboUnidadeGestora"]').click()
self.esperar_elementos_carregarem(By.CSS_SELECTOR, "select[name='cboConta']").click()

self.vars['conta_contabil'] = [cbo.get_attribute('Value') for cbo in self.esperar_elementos_carregarem(By.CSS_SELECTOR, "select[name='cboConta'] option[value^='1232105']", True, 20)]
self.esperar_elementos_carregarem(By.CSS_SELECTOR, "select[name='cboConta']").send_keys(self.vars['conta_contabil'][0])
self.navegador.find_element(By.CSS_SELECTOR, 'select[name="cboMes"]').send_keys('01 - Janeiro  -  Mes Fechado')
if not self.navegador.find_element(By.ID, 'ssoOpcao').is_selected(): self.navegador.find_element(By.ID, 'ssoOpcao').click()
para(1)
self.navegador.find_element(By.ID, 'btnProcurar').click()

ccs_ = self.navegador.find_elements(By.CSS_SELECTOR, '[class="conta-corrente_"]')
ccs = self.navegador.find_elements(By.CSS_SELECTOR, '[class="conta-corrente"]')

ccs.extend(ccs_)


# Imprime a balancete


def tabela_print(pl, aba, salvar=False):
    print(aba.max_column, aba.max_row)
    for j in range(1, aba.max_row + 1):
        for i in range(1, aba.max_column + 1):
            print(aba.cell(row = j, column = i).value, end='     ')
        print('') 

    if salvar: pl.save('tt.xlsx')

In [ ]:
from openpyxl import load_workbook, Workbook
import pandas as pd

excel = load_workbook(r"C:\Users\luan.pinto\Desktop\Códigos\Projeto - Análises Simplificadas para o DAF\123210500 - TESTE 2014.xlsx")
_123210500 = excel.active

# Ler a tabela original e armazenar os dados em um DataFrame do pandas
dados = pd.read_excel(r"C:\Users\luan.pinto\Desktop\Códigos\Projeto - Análises Simplificadas para o DAF\123210500 - TESTE 2014.xlsx")
print('Carrega as planilhas')
# display(dados)
dados.head(10)

# 2. Na linha onde está escrito "Saldo Anterior" escrever na coluna data: 01/01/2014
dados.loc[dados['D/C'] == 'Saldo Anterior', 'Data'] = '01/01/2014'

# 3. Na linha onde está escrito "Saldo Atual" escrever na coluna data: 31/12/2014
dados.loc[dados['D/C'] == 'Saldo Atual', 'Data'] = '31/12/2014'

# 1. Mover as células que contenham "Saldo Anterior" e/ou "Saldo Atual" para a coluna "D/C"
dados['D/C'] = dados.apply(lambda row: row['Movimento'] if 'Saldo A' not in str(row['Movimento']) else '', axis=1)

# Exibir a tabela atualizada
# print(dados)

print('Configura as planilhas')
dados.head(15)
# display(dados)


In [ ]:
# import pandas as pd

# # Criar um DataFrame de exemplo
# dados = pd.DataFrame({'Coluna1': [10, 20, 30],
#                       'Coluna2': [40, 50, 60],
#                       'Coluna3': [70, 80, 90],
#                       'Coluna4': [None, None, None]})

# # Imprimir o DataFrame original
# print("DataFrame original:")
# print(dados)

# # Mover o valor da célula (linha, coluna) para a nova coluna desejada
# dados.iloc[0, 1] = dados.iloc[0, 0]

# # Remover o valor da célula original
# dados.iloc[0, 0] = None

# # Imprimir o DataFrame modificado
# print("\nDataFrame modificado:")
# print(dados)

import pandas as pd

dados = pd.DataFrame({'Coluna1': [10, 20, 30],
                      'Coluna2': [40, 50, 60],
                      'Coluna3': [70, 80, 90],
                      'Coluna4': [100, 100, 100]})

print("DataFrame original:")
print(dados)

# dados['Coluna2'][0], dados['Coluna1'][0] = dados['Coluna1'][0], None

# dados.loc[dados['D/C'] == 'Saldo Anterior', 'Data'] = '01/01/2014'
# dados.loc[dados['Coluna3'] == 80, 'Coluna1'] = 'encontrado'
# dados.loc[dados['Coluna3'] == 80, 'Coluna4'] = 'encontrado'

valor_a_encontrar = '70'

dados['Coluna4'] = dados.apply(lambda row: row['Coluna3'] if   in str(row['Coluna3']) else row['Coluna4'], axis=1)

# Deletar o valor 30 da tabela
# dados = dados[dados['Coluna3'] != 70]
dados = dados.replace(70, None)

print("\nDataFrame modificado:")
print(dados)

